In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tempfile import NamedTemporaryFile
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgb
from matplotlib.patches import Ellipse
from PIL import Image
from astropy.io import fits
from scipy.ndimage.filters import gaussian_filter
from skimage import img_as_float
from skimage.transform import rotate, rescale
from skimage.measure import compare_ssim as ssim
import astropy.units as u
from gzbuilderspirals import deprojecting as dpj
from gzbuilderspirals import getDrawnArms, deprojectArm, rThetaFromXY, xyFromRTheta
from gzbuilderspirals.galaxySpirals import GalaxySpirals
import sdssCutoutGrab as scg
import createSubjectsFunctions as csf

ImportError: No module named 'sdss_psf'

In [3]:
nsa = fits.open('../../subjectUpload/nsa_v1_0_1.fits')

FileNotFoundError: [Errno 2] No such file or directory: '../../subjectUpload/nsa_v1_0_1.fits'

In [4]:
nsa_keys = [
    'NSAID', 'ISDSS', 'INED', 'RA', 'DEC', 'Z', 'SERSIC_BA', 'SERSIC_PHI', 'PETRO_THETA', 
    'IAUNAME', 'ZDIST', 'RUN', 'CAMCOL', 'FIELD', 'RERUN',
]
nsaRas = nsa[1].data['ra']
nsaDecs = nsa[1].data['dec']

df_nsa = pd.DataFrame(
    {key: nsa[1].data[key].byteswap().newbyteorder() for key in nsa_keys}
)

NameError: name 'nsa' is not defined

In [5]:
classifications = pd.read_csv('../classifications/galaxy-builder-classifications_24-7-18.csv')
subjects = pd.read_csv('../classifications/galaxy-builder-subjects_24-7-18.csv')
null = None
true = True
false = False

FileNotFoundError: File b'../classifications/galaxy-builder-classifications_24-7-18.csv' does not exist

In [6]:
montages = [f for f in os.listdir('montageOutputs') if not f[0] == '.']
montageCoordinates = np.array([
    [float(j) for j in i.replace('+', ' ').split(' ')]
    if '+' in i
    else [float(j) for j in i.replace('-', ' -').split(' ')]
    for i in [f for f in os.listdir('montageOutputs') if not f[0] == '.']
])

FileNotFoundError: [Errno 2] No such file or directory: 'montageOutputs'

In [7]:
subjectId = 20902083

In [8]:
meta = eval(subjects[subjects['subject_id'] == subjectId].iloc[0]['metadata'])

NameError: name 'subjects' is not defined

In [9]:
try:
    gal = df_nsa[df_nsa['NSAID'] == int(meta['NSA id'])]
except KeyError:
    gal = {}
    raise KeyError('Metadata does not contain valid NSA id (probably an older galaxy)')
    
gal

NameError: name 'df_nsa' is not defined

In [10]:
montagesDistanceMask = np.add.reduce(
    (montageCoordinates - [gal['RA'].iloc[0], gal['DEC'].iloc[0]])**2,
    axis=1
) < 0.01
usingMontage = np.any(montagesDistanceMask)

NameError: name 'montageCoordinates' is not defined

In [11]:
if usingMontage:
    montageFolder = montages[
        np.where(montagesDistanceMask)[0][0]
    ]
    fitsName = '{}/{}/{}'.format(
        os.path.abspath('montageOutputs'),
        montageFolder,
        'mosaic.fits'
    )
    print('\t- USING MONTAGED IMAGE')
else:
    fitsName = 'fitsImages/{0}/{1}/frame-r-{0:06d}-{1}-{2:04d}.fits'.format(
        int(gal['RUN']),
        int(gal['CAMCOL']),
        int(gal['FIELD'])
    )
print(fitsName)

NameError: name 'usingMontage' is not defined

In [12]:
w = dpj.createWCSObject(gal, 512)
# wcsAngle = dpj.getAngle(gal, w, np.array([512, 512]))
# print('angle:', wcsAngle)
minor, major = dpj.getEllipse(gal, w, np.array([512, 512]))
print(minor, major)
plt.plot(*minor.T)
plt.plot(*major.T)

NameError: name 'gal' is not defined